In [1]:
pip install beautifulsoup4 requests pandas lxml fake_useragent

Note: you may need to restart the kernel to use updated packages.


In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import random
import re
import csv

# Amazon URL for Data Engineering Books
URL = "https://www.amazon.com/s?k=data+engineering+books"

# User-Agent list to avoid detection
USER_AGENTS = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.0.0 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.83 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36"
]

# Headers to mimic a real browser request
HEADERS = {
    "User-Agent": random.choice(USER_AGENTS),
    "Accept-Language": "en-US,en;q=0.5"
}

def get_publication_date_from_product_page(book_url):
    """Fetch the publication date from a book's product page."""
    try:
        product_response = requests.get(book_url, headers=HEADERS)
        if product_response.status_code == 200:
            product_soup = BeautifulSoup(product_response.text, "html.parser")
            possible_locations = [
                "#detailBullets_feature_div",
                "#productDetailsTable",
                "#prodDetails"
            ]
            for location in possible_locations:
                details = product_soup.select_one(location)
                if details:
                    detail_text = details.get_text(strip=True)
                    date_match = re.search(r"(January|February|March|April|May|June|July|August|September|October|November|December) \d{1,2}, \d{4}", detail_text)
                    if date_match:
                        return date_match.group()
            
            # If no full date found, search for a year
            year_match = re.search(r"(19|20)\d{2}", product_soup.text)
            if year_match:
                return year_match.group()
    except Exception as e:
        print(f"Error fetching product page: {e}")
    return None

# Fetch main search results page
response = requests.get(URL, headers=HEADERS)

if response.status_code == 200:
    soup = BeautifulSoup(response.text, "html.parser")
    titles, authors, pub_dates, ratings, prices = [], [], [], [], []
    books = soup.find_all("div", {"data-component-type": "s-search-result"})

    for book in books[:25]:
        title_tag = book.find("h2", class_="a-size-base-plus a-spacing-none a-color-base a-text-normal")
        title = f'"{title_tag.text.strip()}"' if title_tag else None  # Enclose titles in double quotes

        # Extract author information
        author_tag = book.find("div", class_="a-row a-size-base a-color-secondary")
        if author_tag:
            author_links = author_tag.find_all("a")
            author = ", ".join([a.text.strip() for a in author_links]) if author_links else None
        else:
            author = None

        # Extract publication date
        pub_date = None
        date_spans = book.find_all("span", class_="a-size-base a-color-secondary")
        for span in date_spans:
            span_text = span.get_text(strip=True)
            date_match = re.search(r"(January|February|March|April|May|June|July|August|September|October|November|December) \d{1,2}, \d{4}", span_text)
            if date_match:
                pub_date = date_match.group()
                break
        
        if pub_date is None:
            year_match = re.search(r"(19|20)\d{2}", book.text)
            if year_match:
                pub_date = year_match.group()

        # Extract rating
        rating_tag = book.find("span", class_="a-icon-alt")
        rating = rating_tag.text.strip().split()[0] if rating_tag else None

        # Extract price
        price_tag = book.find("span", class_="a-price-whole")
        if not price_tag:
            price_tag = book.find("span", class_="a-offscreen")
        
        if price_tag:
            price = price_tag.text.strip()
            price = price.replace("$", "").replace(",", "")  # Ensure numeric format
        else:
            price = None

        # Extract book URL for further details if needed
        book_url_tag = book.find("a", class_="a-link-normal s-no-outline")
        book_url = "https://www.amazon.com" + book_url_tag["href"] if book_url_tag else None

        # Fetch publication date from product page if missing
        if pub_date is None and book_url:
            pub_date = get_publication_date_from_product_page(book_url)

        # Append extracted data
        titles.append(title)
        authors.append(author)
        pub_dates.append(pub_date)
        ratings.append(rating)
        prices.append(price)

        # Respect Amazon scraping guidelines by adding delay
        time.sleep(random.uniform(1, 3))

    # Create DataFrame
    df = pd.DataFrame({
        "Title": titles,
        "Author": authors,
        "Publication_Date": pub_dates,
        "Rating": ratings,
        "Price": prices
    })

    # Save data to CSV with UTF-8 encoding and comma delimiter
    df.to_csv("amazon_books_data.csv", index=False, encoding="utf-8", sep=",", quoting=csv.QUOTE_MINIMAL)
    
    print("\nData successfully saved to 'amazon_books_data.csv'.")
else:
    print("Failed to retrieve webpage. Amazon may have blocked the request.")


Data successfully saved to 'amazon_books_data.csv'.
